# Laymanz Notebooks: Parameter Efficient Fine-Tuning
Author: Ambrose Ling

**What is this notebook about?**

In this notebook, we will go over some of the most fundamental ideas behind parameter efficient fine-tuning, how they work and why they have been a major advancement in the field of computer vision and generative artifical intelligence. We hope that you can walk away capable of leveraging these techniques to fine-tune your own language models or diffusion models.

**What do I need to set up my environment?**

All of our notebooks will only use numpy, pytorch, matplotlib for visualizations. We will not use any other third-party libraries for model development, optimization or anything like that.

**How is this notebook structured?**
1.
2.
3.


**Covered papers in this notebook**
* LoRA
* GaLoRE
* DoRA


(will do after finishing)

# What is Paramter Efficient Fine Tuning

PEFT aims to leverage methods that reduce the cost of fine-tuning neural network models.
Businesses, engineers, researchers may want to fine-tune state of the art language models or vision models to perform very specialized tasks for them. 

However the main bottleneck is that existing fine-tuning methods may involve full network training on these large models, which requires a lot of computational resources.

If I want to fine-tune a model for multiple different tasks, it is very time consuming and expensive.

## How to estimate the computational resources required for your model?
- Lets assume that your model $\theta$ has 1 billion parameters
- The way we would calculate the memory requirement for fine-tuning such a model is as follows
    - Parameters: 1B $\times$ 4 bytes (32 bits)
    - Gradients: 1B $\times$ 4 bytes (32 bits)
    - Optimizer States: 1B  $\times$ 4 bytes (32 bits)$\times$  2 (depending on the optimizer, Adam saves 2 moment statistics per parameter)
    - Activations: can vary depending on the model


In [1]:
#import necessary libraries
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
linear = nn.Linear(10,20)

In [3]:
type(linear)

torch.nn.modules.linear.Linear

## Low Rank Adaptation Fine Tuning (LoRA)

**What is LoRA?**
LoRA stands for Low Rank Adaptation fine-tuning. LoRA parametrizes the weight update matrix $\Delta W$ as a low rank matrix. Authors found 
that update matrices are intrinsically low rank.

**What is the goal of LoRA?**

The main goal of LoRA is to enable fine-tuning of primiarly Large Language Models (also Diffusion Models) in a parameter efficient way,
meaning without having to fine-tune or update all the weights of our model. This is desirable as this can reduce the computational cost when fine-tuning 
Large Language Models or Diffusion Models on downstream tasks

**How does it work?**
* Assume we have a pretrained model $\theta$ where it has linear layers with weights $W_0$:
* when we fine-tune our model, we freeze $W_0$, which prevents the pretrained weights from having gradient updates
* LoRA decomposes the weight update matrix into 2 matrices $A \in R^{r \times k}$ and $B \in R^{d \times r}$
* where we select $r$ , which is the rank and $r$ should be much smaller than $k$ or $d$
* The paper showed that finetuning perforamnce using a small $r$ is on par with a large $r$ that would be used in full-finetuning 
* This shows that increasing $r$ does not help the weight update matrix cover more meaningful subspaces
* The update formula during fine-tuning is as follows:
    - $xW_{updated}$ = $xW_0$ + $x\Delta W$
    - $xW_{updated}$ = $xW_0$ + $x(A \cdot B)\cdot\frac{\alpha}{r}$
    - where $r$ refers to the rank of low rank parametrization, $$


In [ ]:
# Lets see analyze the dimensionality of the weight matrices
d = 100 # rows 
k = 200 # columns

r = 64 # the intrinsic low rank
W_0 = torch.randn(d,k) 

A = torch.randn(r,k)
B = torch.randn(d,r)

x = torch.randn(k,k)

W_0 = W_0 
delta_W = (A @ B) @ x

assert W_0.shape == delta_W.shape

y = W_0 + delta_W 


## Why does it work?
- It works because of this: there exists an intrinsic dimension
    - Intrinsic dimension means the **minimum number of parameters required to achieve good performance**
    - It also means the **lowest dimensional subspace** we can optimize our objective function in
    - We fine-tune the model using this formula:
    $$
    \theta^d = \theta_0^D + P(\theta^d)
    $$
    where:
    * D is some higher dimension, or original dimensionality of the pretrained parameters
    * d is some lower dimension, the dimensionality we want to perform optimization in for our 
    * P is the projection function or matrix that transforms the lower dimensional parameters to original dimensionality


<center><img src="https://miro.medium.com/v2/resize:fit:1400/1*Ckp6US9r8iDrEP9jW3m3VA.png" ></center>

In [ ]:
class LoRA(nn.Module):
    def __init__(self,rank,scale,layer):
        self.rank = rank
        self.scale = scale
        self.a = nn.Parameter(torch.randn(10,self.rank))
        self.b = nn.Parameter(torch.randn(self.rank,self.rank))
        nn.init.normal(self.a.weight)
        nn.init.normal(self.a.weight)
        nn.init.zero(self.b.weight)
        nn.init.zero(self.b.weight)
    def forward(self,x):
        x = x + self.a @ self.b *(self.scale/self.rank)
        return x




In [ ]:
# SVD decomposition & LoRA

In [ ]:
# Applying LoRA to a pretrained language model



## Gradient Low Rank Adaptation Fine Tuning (GaLoRE)

**What is GaLoRE?**
GaLoRE is a paper released recently aiming to tackle certain limitations of vanilla LoRA
GaLoRE stands for gradient low-rank projection, which is a training method that allows you to still 
train with full parameters but is more **memory efficient** 


**Some of the main problems with LoRA**
- LoRA is not able to reach good performance compared to full-parameter fine-tuning
    - cause 1: the LoRA reparametrization changes the training dynamics
    - cause 2: the optimal weight matrices are not low rank

**How does it work?**
- GaLoRE is used to fine-tune pretrained langauge models
- GaLoRE approximates the gradient matrices as low rank rather than the paramter matrices (gradient matrices show to have slowly changing low-rank structure)
- We compute 2 **projection matrices** $P \in R^{m \times r}$ and $Q \in R^{n \times r}$ to project gradient matrices to **low rank form**, $P^TGQ$
- Only very infrequent updates applied to the projection matrices
- GaLoRE aims to reduce the gradient statistics of both first-order and second-order

## Deep dive into how it works

In full rank training, we represent the weight update rule to be:
$$
W_T = W_0 + n \Sigma_{t=0}^{T-1} \tilde{G_t} = W_0 + n \Sigma_{t=0}^{T-1} \rho_t (\tilde{G_t})
$$

* $W_T$ is weight matrix after $T$ training steps
* $W_0$ is the inital weight matrix
* $ \tilde{G_t}$ is the final gradient matrix to be added to the weight matrix.
* $\rho_t$ is stateful gradient regularizer (optimizer)


In low-rank training, we represent the weight update rule to be:
$$
W_T = W_0 + B_{T}A_{T}
$$
* $B \in R^{m \times r}$ is the low rank gradient matrix (low rank adaptors)
* $A \in R^{r \times n}$ is the low rank projection matrix (low rank adaptors)

In GaLore, we represent tje weight update rule to be:
$$
W_T = W_0 + n \Sigma_{t=0}^{T-1} \tilde{G_t}, \tilde{G_t} = P_t\rho_t(P_t^TG_tQ_t)Q_t^T
$$
* $P_t,Q_t \in R^{m \times r}, \in R^{n \times r}$ are projection matrices

## Memory consumption comparison
In full rank training:
- Adam: $M,V \in R^{m \times n}$  
- Gradient matrix: $G \in R^{m \times n}$
- Weight matrix: $W \in R^{m \times n}$
Total: $3mn$

In LoRA training:
- Adam: $M \in R^{m \times r},V \in R^{n \times r}$ for $A$,  $M \in R^{m \times r},V \in R^{n \times r}$ for $B$
- Gradient matrix: $G \in R^{m \times n}$
- Weight matrix: $W \in R^{m \times n}$, $A \in R^{m \times r}$,$B \in R^{n \times r}$
Total: $$

In GaLoRE training:
- Adam: $M,V \in R^{n \times r}$  
- Gradient matrix: $G \in R^{n \times r}$
- Weight matrix: $W \in R^{m \times n}$
- Projection matrices: $P \in R^{m \times r}$


**Original Adam Optimizer Algorithm**
<center><a href="https://ibb.co/JK5nkmd"><img src="https://i.ibb.co/xDJh5Xs/Screenshot-2024-06-05-at-1-32-18-PM.png" alt="Screenshot-2024-06-05-at-1-32-18-PM" border="0"></a></center>

In [2]:
# Trainng code for the Adam Optimizer

from torch.optim import Optimizer
import math

class Adam(Optimizer):
    def  __init__(self,params,lr = 1e-6,betas = (0.99,0.9),eps=1e-6,weight_decay=0.0):
        defaults = dict(lr=lr,betas=betas,weight_decay=weight_decay,eps=eps)
        super().__init__(params,defaults=defaults)
    def step(self,closure=None):
        #Iterature through all the parameter 
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                state = self.state[p]
                step_size = group['lr']
                # state is a dictionary that holds all the optimizer configurations for each parameter
                if len(state) ==0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p.data)
                    state['exp_avg_sqr'] = torch.zeros_like(p.data)
                exp_avg = state['exp_avg'] #first moment estimate
                exp_avg_sqr = state['exp_avg_sqr'] # second moment estimate
                beta1,beta2 = group['betas'] # get betas
                exp_avg.mul_(beta1).add_((1-beta1),grad) #update biased first moment estimate
                exp_avg_sqr.mul(beta2.addcmul_((1-beta2)),grad,grad) # update biased second moment estimate
                denom = exp_avg_sqr.sqrt().add_(group['eps'])

                # If there is bias correction
                if group['bias_correction'] == True:
                    bias_corrected_first_moment = 1 - beta1 ** state['step']
                    bias_corrected_second_moment = 1 - beta2 ** state['step']
                    step_size = step_size * math.sqrt(bias_corrected_second_moment) / bias_corrected_first_moment

                # Weight update
                p.data.addcdiv_(-step_size,exp_avg,denom)

**GaLoRE+Adam Optimizer Algorithm**
<center><a href="https://ibb.co/nDD9Sj7"><img src="https://i.ibb.co/gDDNCJS/Screenshot-2024-06-05-at-4-02-38-PM.png" alt="Screenshot-2024-06-05-at-4-02-38-PM" border="0"></a></center>

In [1]:
# Trainng code for the GaLoRE Optimizer (Ambrose's implementation)

from torch.optim import Optimizer
import torch
import math

class GaLoREOptimizer(Optimizer):
    def  __init__(self,params,lr = 1e-6,betas = (0.99,0.9),eps=1e-6,weight_decay=0.0,rank=32,subspace_freq=10,lora_scale=1.0):
        defaults = dict(lr=lr,betas=betas,weight_decay=weight_decay,eps=eps, )
        self.subspace_freq = subspace_freq # corresponds to T, subspace change frequency
        self.lora_rank = rank # corresponds to LoRA rank
        self.scaling_factor = lora_scale
        super(GaLoREOptimizer,self).__init__(params,defaults=defaults)
    def step(self,closure=None):
        #Iterature through all the parameter 

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                #p.data.shape (512,784)
                m = p.grad.data.shape[0] # 512
                n = p.grad.data.shape[-1] # 768
                r = self.lora_rank
                state = self.state[p]
                step_size = group['lr']
                # state is a dictionary that holds all the optimizer configurations for each parameter
                if len(state) ==0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros(r,n)
                    state['exp_avg_sqr'] = torch.zeros(r,n)
                    state['projection'] = torch.zeros(m,r)
                if state['step'] % self.subspace_freq == 0:
                    U,_,_ = torch.svd(grad)
                    #                      m x r
                    state['projection'] = U[:,:r] #state['projection'].shape (512,32)
                else:
                    pass

                # Project the gradient matrix to low rank (compact space)
                r_t = state['projection'].T @ grad # (32,512) * (512,784) = (32, 784) = r x n
                # Exponential moving average of **low-rank projection** of gradient values
                exp_avg = state['exp_avg'] #first moment estimate
                # Exponential moving average of the square of the **low-rank projection** of gradient values
                exp_avg_sqr = state['exp_avg_sqr'] # second moment estimate
                beta1,beta2 = group['betas'] # get betas
                exp_avg.mul_(beta1).add_((1-beta1),r_t) #update biased first moment estimate
                exp_avg_sqr.mul(beta2).addcmul_((1-beta2),r_t,r_t) # update biased second moment estimate
                denom = exp_avg_sqr.sqrt().add_(group['eps']) # denom.shape (32,784)

                # If there is bias correction
                if 'bias_correction' in group:
                    bias_corrected_first_moment = 1 - beta1 ** state['step']
                    bias_corrected_second_moment = 1 - beta2 ** state['step']
                    step_size = step_size * math.sqrt(bias_corrected_second_moment) / bias_corrected_first_moment
                n_t = exp_avg / denom 

                # Project low-rank graident matriz back t to original vectior subspace
                #                               m x r           r x n
                g_t = self.scaling_factor * state['projection'] @ n_t
                state['step'] += 1
                # Weight update
                p.data.add_(-step_size*g_t)

## Weight Decomposition Low Rank Adaptation Fine Tuning (DoRA)

**What is DoRA?**


**How does it work?**

## Q-LoRA (Quantized Fine-Tuning) 
 
## What is Q-LoRA?
Q-LoRA is a parameter efficient fine tuning method that aims to **reduce memory footprint** of training large models (specifically LLMs)
using some of the following innovations:
-  4-bit NormalFloat (NF4)
- Double quantization (quantizing the quantization constants)
- Paged optimizers (reduce memory spikes)

**NOTE**:
- QLoRA data storage data type: nf4
- QLoRA computation data type: bf16

For 1 linear layer, Q-LoRA:
$$
Y^{bf16} = X^{bf16} \text{double quantize}(c^{fp32}_1,c^{k-bit}_2,W^{NF4}) + X^{bf16} A^{bf16} B^{bf6}
$$



## How does it work?

**Block wise k-bit quantization**


Quantization:
$$
x_{quantized} = X^{int8} = round(\frac{127}{absmax(X^{fp32})}) = round(c^{fp32},X^{fp32})
$$
Dequantization: 
$$
X_{dequantized} = X^{fp32} = \frac{X^{int8}}{c^{fp32}}
$$


* We chunk the tensor $X \in R^{b \times h}$ into $N$ contigous blocks of size $B$ 
* We will get $n = (b \times h) / B$ number of blocks
* We quantize these blocks indepdently
* We will get $n$ quantization constants with this quantized tensor

**4-bit NormalFloat Quantization**
- Builds on Quantile Quantization: 
    - Each quantization bin has an equal number of values assigned from input tensor
    - It estimates the quantile of the input tensor through the empirical cummulative distribution function
- Limitations with Quantile Quantization:
    - Quantile Estimation algorithms have large quantization error
    - Quantile Estimation is expensive
- Input tesnors come from a distribution fixed up to a quantization constant -> Input tensors have the same quantiles

Approach
- 1) Estimate the $2^{k}+1$ quantiles of a theoretical N(0,I) distribution to obtain a $k$ - bit quantile quantization data type for normal distirbution
- 2) Take this data type and normalize its values into the $[-1,1]$ range.
- 3) Quantize an input weight tensor by normalizing it into the $[-1,1]$ range through absolute maximum rescaling

How do we estimate the $2^k$ values $q_i$ of the data type as follows?
$$
q_i = \frac{1}{2} (Q_X(\frac{i}{2^k + 1}) + Q_X(\frac{i+1}{2^k +1}))
$$
where
* $i$ refers to the index into the sorted array of a tensor
* $Q_{X}$ is the quantile function (or inverse CDF)

We create asymmetric data type by estimating the quantiles $q_i$ of 2 ranges:
- $2^{k-1}$ for neagitve part
- $2^{k-1} + 1$ for the positive part


**Double Quantization**
- We firther quantize the quantization constants $c^{fp32}$ (constant of the 1st quantization) as input to second quantization
- Memory comparison:
    - Original quantization: 32 bit constants, block size 64 => 32/64 = 0.5 bits per parameter ($n = (b \times h) / B$, where $B$ is 64)
        - Intuition: 
            - For every 64 parameters you have a quantization constant added that is size 32 bit, so per parameter, you are adding 0.5 bits
    - Double quantization: 8 bit floats, block size 256 in 2nd quantization
        - $8/64 + 32/(64*256) = 0.127$ bits  
            - Intuition: 
                - In original quantization, you have blocks of size 64, each block **used** to be represented by 1 quantization constant. 
                - In double quantization, you look at 256 of these blocks of 64,so you have 256 of these quantizaiton constants. You then perform quantization on the 256 constants, they are then turned to 8 bits. As a result you get 1 32bit number for every 256 blocks.
     


- Input: $c_2^{FP32}$
- Output: $c_2^{FP8}$ and $c_1^{FP32}$
where 
* $c_2^{FP8}$ is the quantized quantization constant
* $c_1^{FP32}$ is the second level quantization constant


**Paged Optimizers**
- use paging feature to evict optimizer states back to CPU RAM if GPU RAM isnt enough
- page it back into GPU when needed for gradient step


**NOTE:** stuff about quantiles and estimating quantiles

* A quantile is a statistical measure that divies the distribution into equal subgroups
* A quantile can be:
    - Quartiles: Divide the data into 4 equal parts
    - Deciles: Divide into 10 equal parts
    - Percentiles: Divide into 100 equal parts
* How to find the quantile in a distribution?
    - 1) order the data in ascending order
    - 2) calculate the position of the **desired quantile** : Position = number of total elements or observations $\times$ the desired quantile (as a fraction)
    - 3) interpolate to estimate the quantile value if the position is not an integer
* More about CDFs and quantiles:
    - CDF: returns the probabilities of $X$ being smaller than or equal to some value $x$, $Pr(X \leq x) = F(x) = p$
    - Inverse CDF (quantile function): $F^{-1}(p) = x$ 
    <center>
    <img src="https://i.sstatic.net/SNViH.png"/>
    </center>

In [30]:
#
import torch

# We want to store the weights as 4-bit integers
# We create 16 levels spaced btw -1 and 1
quantiles = torch.linspace(-1,1,16)

print(quantiles)

# This is our data tensor
tensor_fp32 = torch.tensor(0.567)

print(torch.abs(quantiles - tensor_fp32))

# We find the floating point number that is closest to 0.567 (which is 0.6), and we will
# quantize it to that float 0.6
index = torch.argmin(torch.abs(quantiles - tensor_fp32))

# All we need to store is the index, which is just an integer (13)

# Our dequantization error:
error = 0.6 - 0.567



tensor([-1.0000, -0.8667, -0.7333, -0.6000, -0.4667, -0.3333, -0.2000, -0.0667,
         0.0667,  0.2000,  0.3333,  0.4667,  0.6000,  0.7333,  0.8667,  1.0000])
tensor([1.5670, 1.4337, 1.3003, 1.1670, 1.0337, 0.9003, 0.7670, 0.6337, 0.5003,
        0.3670, 0.2337, 0.1003, 0.0330, 0.1663, 0.2997, 0.4330])
tensor(12)


##

In [ ]:
# Double quantization

In [48]:
# 4-bit Normal Float 
nf4 = torch.tensor([-1.0, -0.6961928009986877, -0.5250730514526367, -0.39491748809814453, -0.28444138169288635, -0.18477343022823334, -0.09105003625154495, 0.0, 0.07958029955625534, 0.16093020141124725,
0.24611230194568634, 0.33791524171829224, 0.44070982933044434, 0.5626170039176941, 0.7229568362236023, 1.0])
print(len(nf4))

16


In [86]:
# Step 1: Create some weight tensor

x = torch.rand(100,100)
x

tensor([[0.3607, 0.4957, 0.1694,  ..., 0.6392, 0.8512, 0.1933],
        [0.8317, 0.2010, 0.4518,  ..., 0.1055, 0.3967, 0.1426],
        [0.9364, 0.7875, 0.5563,  ..., 0.3970, 0.6429, 0.7989],
        ...,
        [0.6362, 0.1420, 0.4050,  ..., 0.6945, 0.0891, 0.6628],
        [0.3368, 0.7009, 0.2769,  ..., 0.5620, 0.5264, 0.5288],
        [0.6726, 0.1954, 0.0083,  ..., 0.5836, 0.2798, 0.3326]])

In [87]:
# Step 2: normalize it btw -1 and 1
x = (x * 2) -1
x

tensor([[-0.2786, -0.0086, -0.6613,  ...,  0.2783,  0.7024, -0.6133],
        [ 0.6634, -0.5979, -0.0963,  ..., -0.7889, -0.2066, -0.7149],
        [ 0.8729,  0.5751,  0.1126,  ..., -0.2060,  0.2858,  0.5978],
        ...,
        [ 0.2724, -0.7159, -0.1900,  ...,  0.3891, -0.8218,  0.3256],
        [-0.3265,  0.4018, -0.4461,  ...,  0.1240,  0.0527,  0.0576],
        [ 0.3453, -0.6092, -0.9833,  ...,  0.1672, -0.4404, -0.3349]])

In [89]:
# Step 3: Perform the quantization
x_quantized = (((8 - 1) / x.abs().max())*x).round().to(torch.int8)
c = ((8 - 1) / x.abs().max())
print(x_quantized)
print(x_quantized.min())

tensor([[-2,  0, -5,  ...,  2,  5, -4],
        [ 5, -4, -1,  ..., -6, -1, -5],
        [ 6,  4,  1,  ..., -1,  2,  4],
        ...,
        [ 2, -5, -1,  ...,  3, -6,  2],
        [-2,  3, -3,  ...,  1,  0,  0],
        [ 2, -4, -7,  ...,  1, -3, -2]], dtype=torch.int8)
tensor(-7, dtype=torch.int8)


In [91]:
x_quantized = x_quantized+8

In [93]:
x_quantized.max()

tensor(15, dtype=torch.int8)

In [76]:
x_dequantized  = (x_quantized / ((16 -1)/x.abs().max())).float()
x_dequantized

tensor([[ 0.6250,  0.4375,  1.0000,  ..., -0.1875,  0.2500,  0.2500],
        [-0.2500,  0.5000, -0.8750,  ...,  0.0000,  0.2500, -0.5000],
        [-0.3750,  0.2500, -0.6250,  ..., -0.2500, -0.8750, -0.3750],
        ...,
        [-0.3750, -0.2500,  0.1250,  ...,  0.0000,  0.0000,  0.4375],
        [ 0.3125,  0.3750,  0.6875,  ...,  0.3750, -0.6250,  0.6250],
        [-0.7500, -0.8750,  0.8125,  ..., -0.4375,  0.8750,  0.8750]])